In [1]:
import sys
sys.path.append('../atac_rna/')

import numpy as np
import pickle
import os
import pandas as pd

import time

import scanpy
import numpy as np

import torch
from torch import optim

import model_lord

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import anndata as ad



In [2]:
snl111=ad.read('/data/xinyi/c2p/citeseq_totalvi/totalVI_reproducibility/data/spleen_lymph_111.h5ad')

In [3]:
np.sum(snl111.var['hvg_encode'])

4005

In [5]:
#same preprocessing as totalVI for proteins and highly variable genes
snl111_rna=snl111[:,snl111.var["hvg_encode"]]

In [6]:
rna=snl111_rna.X.toarray()
rna

array([[ 0.,  0.,  0., ...,  0., 22.,  2.],
       [ 0.,  0.,  0., ...,  0., 19.,  3.],
       [ 0.,  0.,  0., ...,  0.,  7.,  2.],
       ...,
       [ 2.,  0.,  0., ...,  0.,  6.,  0.],
       [ 0.,  0.,  0., ...,  0., 11.,  0.],
       [ 0.,  0.,  0., ...,  0., 36.,  0.]], dtype=float32)

In [7]:
#train-test split
np.random.seed(3)
pctVal=0.05
pctTest=0.1

allIdx_all=np.arange(rna.shape[0])
np.random.shuffle(allIdx_all)
valIdx_all=allIdx_all[:int(pctVal*rna.shape[0])]
testIdx_all=allIdx_all[int(pctVal*rna.shape[0]):(int(pctVal*rna.shape[0])+int(pctTest*rna.shape[0]))]
trainIdx_all=allIdx_all[(int(pctVal*rna.shape[0])+int(pctTest*rna.shape[0])):]



In [8]:
#preprocess rna
rna_posweight=(rna[trainIdx_all].size-np.sum(rna[trainIdx_all]))/np.sum(rna[trainIdx_all])
#preprocess
rna=np.log(rna+1/2)
rna=(rna-np.min(rna,axis=1,keepdims=True))/(np.max(rna,axis=1,keepdims=True)-np.min(rna,axis=1,keepdims=True))


In [9]:
#preprocess protein
keep_pro_111 = np.array(
    [not p.startswith("HTO") for p in snl111.uns["protein_names"]]
)

snl111.obsm["protein_expression"] = snl111.obsm["protein_expression"][
    :, keep_pro_111
]
snl111.uns["protein_names"] = snl111.uns["protein_names"][keep_pro_111]

In [10]:
protein=np.array(snl111.obsm["protein_expression"])

In [11]:
protein.shape

(16828, 110)

In [12]:
#preprocess protein
protein_posweight=(protein[trainIdx_all].size-np.sum(protein[trainIdx_all]>0))/np.sum(protein[trainIdx_all]>0)
#preprocess
protein=np.log(protein+1/2)
protein=(protein-np.min(protein,axis=1,keepdims=True))/(np.max(protein,axis=1,keepdims=True)-np.min(protein,axis=1,keepdims=True))


In [24]:
log_data=True
normalize='minmax'
hiddenSize=1024
sharedSize=50
dSpecificSize=30
seed=1


In [16]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["CUDA_LAUNCH_BLOCKING"] = "2"
 

batchsize=512
saveFreq=100
epochs=5000
lr_latent=0.001
lr_decoder=0.0001
weight_decay=0

dropout=0.01
XreconWeight=20
ridgeL=0.01

addNoise=True
actDecay=0.001

testSaveName='lord_bce'
name='test1_noSharedRecon_bothdays'
logsavepath=os.path.join('/data/xinyi/c2p/citeseq_totalvi/results/log',testSaveName,name)
modelsavepath=os.path.join('/data/xinyi/c2p/citeseq_totalvi/results/models',testSaveName,name)
plotsavepath=os.path.join('/data/xinyi/c2p/citeseq_totalvi/results/plots',testSaveName,name)

loss_x_rna=torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor(rna_posweight).cuda())
loss_x_protein=torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor(protein_posweight).cuda())

In [17]:
if not os.path.exists(os.path.join('/data/xinyi/c2p/citeseq_totalvi/results/plots',testSaveName)):
    os.mkdir(os.path.join('/data/xinyi/c2p/citeseq_totalvi/results/plots',testSaveName))
    os.mkdir(os.path.join('/data/xinyi/c2p/citeseq_totalvi/results/models',testSaveName))
    os.mkdir(os.path.join('/data/xinyi/c2p/citeseq_totalvi/results/log',testSaveName))
if not os.path.exists(logsavepath):
    os.mkdir(logsavepath)
if not os.path.exists(modelsavepath):
    os.mkdir(modelsavepath)
if not os.path.exists(plotsavepath):
    os.mkdir(plotsavepath)
nFeatures_rna=rna.shape[1]
nFeatures_protein=protein.shape[1]

In [20]:
def train(epoch):
    t = time.time()
    model_rna.train()
    model_protein.train()
    
    loss_all=0
    loss_x_train_all_rna=0
    loss_x_train_all_protein=0
    loss_lNorm_all=0
    loss_lNorm_all_rna=0
    loss_lNorm_all_protein=0
    
    ntrainBatches=int(np.ceil(trainIdx_all.shape[0]/batchsize))
    for i in range(ntrainBatches):
        trainIdx=trainIdx_all[i*batchsize:min((i+1)*batchsize,trainIdx_all.shape[0])]
        traintarget_rna=torch.tensor(rna[trainIdx]).cuda().float()
        traintarget_protein=torch.tensor(protein[trainIdx]).cuda().float()
        trainIdx=torch.tensor(trainIdx)
        trainInput_shared=latent_shared(trainIdx).cuda().float()
        trainInput_rna=latent_rna(trainIdx).cuda().float()
        trainInput_protein=latent_protein(trainIdx).cuda().float()

        if addNoise:
            trainInput_shared=trainInput_shared+torch.randn_like(trainInput_shared)
            trainInput_rna=trainInput_rna+torch.randn_like(trainInput_rna)
            trainInput_protein=trainInput_protein+torch.randn_like(trainInput_protein)
            
        optimizer_rna.zero_grad()
        optimizer_latent.zero_grad()
        optimizer_protein.zero_grad()

        recon_rna= model_rna(torch.cat((trainInput_shared,trainInput_rna),dim=1))
        protein_recon = model_protein(torch.cat((trainInput_shared,trainInput_protein),dim=1))

        loss_x_train_rna=loss_x_rna(recon_rna, traintarget_rna)
        loss_x_train_protein=loss_x_protein(protein_recon, traintarget_protein)        
        loss=loss_x_train_rna+loss_x_train_protein
        
        if addNoise:
            loss_lNorm=torch.mean(torch.linalg.norm(latent_shared(trainIdx).cuda(),dim=1))/latent_shared.embedding_dim
            loss_lNorm_all+=loss_lNorm.item()
            loss+=loss_lNorm*actDecay
            
            loss_lNorm_rna=torch.mean(torch.linalg.norm(latent_rna(trainIdx).cuda(),dim=1))/latent_rna.embedding_dim
            loss_lNorm_all_rna+=loss_lNorm_rna.item()
            loss+=loss_lNorm_rna*actDecay
            
            loss_lNorm_protein=torch.mean(torch.linalg.norm(latent_protein(trainIdx).cuda(),dim=1))/latent_protein.embedding_dim
            loss_lNorm_all_protein+=loss_lNorm_protein.item()
            loss+=loss_lNorm_protein*actDecay
            
        
        loss_x_train_all_protein+=loss_x_train_protein.item()
        loss_x_train_all_rna+=loss_x_train_rna.item()
        loss_all+=loss.item()
        
            
        loss.backward()
        optimizer_rna.step()
        optimizer_latent.step()
        optimizer_protein.step()


    loss_x_train_all_protein=loss_x_train_all_protein/ntrainBatches
    loss_x_train_all_rna=loss_x_train_all_rna/ntrainBatches
    loss_all=loss_all/ntrainBatches
    if addNoise:
        loss_lNorm_all=loss_lNorm_all/ntrainBatches
        loss_lNorm_all_rna=loss_lNorm_all_rna/ntrainBatches
        loss_lNorm_all_protein=loss_lNorm_all_protein/ntrainBatches

    
    with torch.no_grad():
        model_rna.eval()
        model_protein.eval()
        loss_x_val_all_rna=0
        loss_x_val_all_protein=0
        loss_val_all=0
        nvalBatches=int(np.ceil(valIdx_all.shape[0]/batchsize))
        for i in range(nvalBatches):
            valIdx=valIdx_all[i*batchsize:min((i+1)*batchsize,valIdx_all.shape[0])]
            valtarget_rna=torch.tensor(rna[valIdx]).cuda().float()
            valtarget_protein=torch.tensor(protein[valIdx]).cuda().float()
            valIdx=torch.tensor(valIdx)
            valInput_shared=latent_shared(valIdx).cuda().float()
            valInput_rna=latent_rna(valIdx).cuda().float()
            valInput_protein=latent_protein(valIdx).cuda().float()

            recon_rna= model_rna(torch.cat((valInput_shared,valInput_rna),dim=1))
            protein_recon = model_protein(torch.cat((valInput_shared,valInput_protein),dim=1))

            loss_x_val_rna=loss_x_rna(recon_rna, valtarget_rna)
            loss_x_val_protein=loss_x_protein(protein_recon, valtarget_protein)


            loss=loss_x_val_rna+loss_x_val_protein
            
            loss_x_val_all_rna+=loss_x_val_rna.item()
            loss_x_val_all_protein+=loss_x_val_protein.item()
            loss_val_all+=loss.item()

        loss_x_val_all_rna=loss_x_val_all_rna/nvalBatches
        loss_x_val_all_protein=loss_x_val_all_protein/nvalBatches
        loss_val_all=loss_val_all/nvalBatches
    
    print(' Epoch: {:04d}'.format(epoch),
          'loss_train: {:.4f}'.format(loss_all),
          'loss_x_train_rna: {:.4f}'.format(loss_x_train_all_rna),
          'loss_x_train_protein: {:.4f}'.format(loss_x_train_all_protein),
          'loss_val: {:.4f}'.format(loss_val_all),
          'loss_x_val_rna: {:.4f}'.format(loss_x_val_all_rna),
          'loss_x_val_protein: {:.4f}'.format(loss_x_val_all_protein),
          'time: {:.4f}s'.format(time.time() - t))
    return loss_all,loss_x_train_all_rna,loss_x_train_all_protein,loss_val_all,loss_x_val_all_rna,loss_x_val_all_protein,loss_lNorm_all,loss_lNorm_all_rna,loss_lNorm_all_protein




In [ ]:
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.enabled = True

model_rna =model_lord.fc_decode_l4(nFeatures_rna,sharedSize+dSpecificSize,hiddenSize, dropout)
model_protein =model_lord.fc_decode_l4(nFeatures_protein, sharedSize+dSpecificSize,hiddenSize, dropout)


model_rna.cuda()
model_protein.cuda()

optimizer_rna = optim.Adam(model_rna.parameters(), lr=lr_decoder, weight_decay=weight_decay)
optimizer_protein = optim.Adam(model_protein.parameters(), lr=lr_decoder, weight_decay=weight_decay)

latent_shared=torch.nn.Embedding(rna.shape[0],sharedSize)
latent_rna=torch.nn.Embedding(rna.shape[0],dSpecificSize)
latent_protein=torch.nn.Embedding(rna.shape[0],dSpecificSize)
optimizer_latent= torch.optim.Adam([latent_shared.weight,latent_rna.weight,latent_protein.weight], lr=lr_latent, weight_decay=weight_decay)

train_loss=[np.inf]*(epochs)
val_loss=[np.inf]*(epochs)

train_loss_rna=[np.inf]*(epochs)
val_loss_rna=[np.inf]*(epochs)

train_loss_protein=[np.inf]*(epochs)
val_loss_protein=[np.inf]*(epochs)

train_lNorm=[np.inf]*(epochs)
train_lNorm_rna=[np.inf]*(epochs)
train_lNorm_protein=[np.inf]*(epochs)

t_ep=time.time()

for ep in range(epochs):

    train_loss[ep],train_loss_rna[ep],train_loss_protein[ep],val_loss[ep],val_loss_rna[ep],val_loss_protein[ep],train_lNorm[ep],train_lNorm_rna[ep],train_lNorm_protein[ep]=train(ep)


    if ep%saveFreq == 0 and ep != 0:
        torch.save(model_rna.cpu().state_dict(), os.path.join(modelsavepath,str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'_ep'+str(ep)+'_rna.pt'))
        torch.save(model_protein.cpu().state_dict(), os.path.join(modelsavepath,str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'_ep'+str(ep)+'_protein.pt'))

        with open(os.path.join(modelsavepath,'latentRNA_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'_ep'+str(ep)), 'wb') as output:
            pickle.dump(latent_rna.weight, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(modelsavepath,'latentShared_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'_ep'+str(ep)), 'wb') as output:
            pickle.dump(latent_shared.weight, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(modelsavepath,'latentprotein_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'_ep'+str(ep)), 'wb') as output:
            pickle.dump(latent_protein.weight, output, pickle.HIGHEST_PROTOCOL)



    model_rna.cuda()
    model_protein.cuda()
    torch.cuda.empty_cache()
print(' total time: {:.4f}s'.format(time.time() - t_ep))





 Epoch: 0000 loss_train: 0.8948 loss_x_train_rna: 0.5342 loss_x_train_protein: 0.3603 loss_val: 0.8196 loss_x_val_rna: 0.4902 loss_x_val_protein: 0.3294 time: 1.3998s
 Epoch: 0001 loss_train: 0.5087 loss_x_train_rna: 0.2999 loss_x_train_protein: 0.2085 loss_val: 0.4827 loss_x_val_rna: 0.2805 loss_x_val_protein: 0.2023 time: 0.7807s
 Epoch: 0002 loss_train: 0.3850 loss_x_train_rna: 0.2220 loss_x_train_protein: 0.1627 loss_val: 0.3731 loss_x_val_rna: 0.2138 loss_x_val_protein: 0.1593 time: 0.7483s
 Epoch: 0003 loss_train: 0.3417 loss_x_train_rna: 0.1934 loss_x_train_protein: 0.1480 loss_val: 0.3366 loss_x_val_rna: 0.1911 loss_x_val_protein: 0.1455 time: 0.7132s
 Epoch: 0004 loss_train: 0.3229 loss_x_train_rna: 0.1803 loss_x_train_protein: 0.1423 loss_val: 0.3210 loss_x_val_rna: 0.1805 loss_x_val_protein: 0.1405 time: 0.7146s
 Epoch: 0005 loss_train: 0.3132 loss_x_train_rna: 0.1732 loss_x_train_protein: 0.1396 loss_val: 0.3142 loss_x_val_rna: 0.1756 loss_x_val_protein: 0.1386 time: 0.6955

 Epoch: 0364 loss_train: 0.4985 loss_x_train_rna: 0.1209 loss_x_train_protein: 0.1276 loss_xShared_train_rna: 0.1218 loss_xShared_train_protein: 0.1279 loss_val: 0.5638 loss_x_val_rna: 0.1447 loss_x_val_protein: 0.1374 loss_xShared_val_rna: 0.1445 loss_xShared_val_protein: 0.1373 time: 0.3417s
 Epoch: 0365 loss_train: 0.4985 loss_x_train_rna: 0.1209 loss_x_train_protein: 0.1276 loss_xShared_train_rna: 0.1217 loss_xShared_train_protein: 0.1279 loss_val: 0.5637 loss_x_val_rna: 0.1445 loss_x_val_protein: 0.1375 loss_xShared_val_rna: 0.1444 loss_xShared_val_protein: 0.1373 time: 0.3530s
 Epoch: 0366 loss_train: 0.4984 loss_x_train_rna: 0.1209 loss_x_train_protein: 0.1276 loss_xShared_train_rna: 0.1217 loss_xShared_train_protein: 0.1279 loss_val: 0.5638 loss_x_val_rna: 0.1446 loss_x_val_protein: 0.1374 loss_xShared_val_rna: 0.1445 loss_xShared_val_protein: 0.1373 time: 0.3255s
 Epoch: 0367 loss_train: 0.4984 loss_x_train_rna: 0.1209 loss_x_train_protein: 0.1276 loss_xShared_train_rna: 0.121

 Epoch: 0392 loss_train: 0.4969 loss_x_train_rna: 0.1202 loss_x_train_protein: 0.1275 loss_xShared_train_rna: 0.1211 loss_xShared_train_protein: 0.1278 loss_val: 0.5641 loss_x_val_rna: 0.1448 loss_x_val_protein: 0.1375 loss_xShared_val_rna: 0.1445 loss_xShared_val_protein: 0.1373 time: 0.3269s
 Epoch: 0393 loss_train: 0.4968 loss_x_train_rna: 0.1202 loss_x_train_protein: 0.1275 loss_xShared_train_rna: 0.1211 loss_xShared_train_protein: 0.1278 loss_val: 0.5642 loss_x_val_rna: 0.1448 loss_x_val_protein: 0.1374 loss_xShared_val_rna: 0.1446 loss_xShared_val_protein: 0.1374 time: 0.3381s
 Epoch: 0394 loss_train: 0.4967 loss_x_train_rna: 0.1201 loss_x_train_protein: 0.1275 loss_xShared_train_rna: 0.1210 loss_xShared_train_protein: 0.1278 loss_val: 0.5640 loss_x_val_rna: 0.1448 loss_x_val_protein: 0.1374 loss_xShared_val_rna: 0.1445 loss_xShared_val_protein: 0.1372 time: 0.3443s
 Epoch: 0395 loss_train: 0.4967 loss_x_train_rna: 0.1201 loss_x_train_protein: 0.1275 loss_xShared_train_rna: 0.121

 Epoch: 0420 loss_train: 0.4955 loss_x_train_rna: 0.1195 loss_x_train_protein: 0.1276 loss_xShared_train_rna: 0.1204 loss_xShared_train_protein: 0.1276 loss_val: 0.5645 loss_x_val_rna: 0.1450 loss_x_val_protein: 0.1375 loss_xShared_val_rna: 0.1448 loss_xShared_val_protein: 0.1373 time: 0.3315s
 Epoch: 0421 loss_train: 0.4955 loss_x_train_rna: 0.1195 loss_x_train_protein: 0.1276 loss_xShared_train_rna: 0.1204 loss_xShared_train_protein: 0.1276 loss_val: 0.5647 loss_x_val_rna: 0.1451 loss_x_val_protein: 0.1376 loss_xShared_val_rna: 0.1448 loss_xShared_val_protein: 0.1373 time: 0.3262s
 Epoch: 0422 loss_train: 0.4954 loss_x_train_rna: 0.1194 loss_x_train_protein: 0.1276 loss_xShared_train_rna: 0.1204 loss_xShared_train_protein: 0.1276 loss_val: 0.5646 loss_x_val_rna: 0.1451 loss_x_val_protein: 0.1375 loss_xShared_val_rna: 0.1448 loss_xShared_val_protein: 0.1373 time: 0.3453s
 Epoch: 0423 loss_train: 0.4953 loss_x_train_rna: 0.1194 loss_x_train_protein: 0.1276 loss_xShared_train_rna: 0.120

 Epoch: 0448 loss_train: 0.4936 loss_x_train_rna: 0.1188 loss_x_train_protein: 0.1272 loss_xShared_train_rna: 0.1198 loss_xShared_train_protein: 0.1275 loss_val: 0.5649 loss_x_val_rna: 0.1453 loss_x_val_protein: 0.1374 loss_xShared_val_rna: 0.1448 loss_xShared_val_protein: 0.1375 time: 0.3354s
 Epoch: 0449 loss_train: 0.4935 loss_x_train_rna: 0.1187 loss_x_train_protein: 0.1272 loss_xShared_train_rna: 0.1198 loss_xShared_train_protein: 0.1275 loss_val: 0.5648 loss_x_val_rna: 0.1452 loss_x_val_protein: 0.1374 loss_xShared_val_rna: 0.1448 loss_xShared_val_protein: 0.1373 time: 0.3285s
 Epoch: 0450 loss_train: 0.4934 loss_x_train_rna: 0.1187 loss_x_train_protein: 0.1272 loss_xShared_train_rna: 0.1197 loss_xShared_train_protein: 0.1275 loss_val: 0.5646 loss_x_val_rna: 0.1451 loss_x_val_protein: 0.1374 loss_xShared_val_rna: 0.1447 loss_xShared_val_protein: 0.1373 time: 0.3254s
 Epoch: 0451 loss_train: 0.4934 loss_x_train_rna: 0.1187 loss_x_train_protein: 0.1272 loss_xShared_train_rna: 0.119

...

 Epoch: 3556 loss_train: 0.4453 loss_x_train_rna: 0.0974 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.0982 loss_xShared_train_protein: 0.1245 loss_val: 0.6023 loss_x_val_rna: 0.1541 loss_x_val_protein: 0.1459 loss_xShared_val_rna: 0.1526 loss_xShared_val_protein: 0.1496 time: 0.4304s
 Epoch: 3557 loss_train: 0.4453 loss_x_train_rna: 0.0974 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.0982 loss_xShared_train_protein: 0.1245 loss_val: 0.6018 loss_x_val_rna: 0.1542 loss_x_val_protein: 0.1459 loss_xShared_val_rna: 0.1527 loss_xShared_val_protein: 0.1491 time: 0.4707s
 Epoch: 3558 loss_train: 0.4453 loss_x_train_rna: 0.0974 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.0982 loss_xShared_train_protein: 0.1245 loss_val: 0.6001 loss_x_val_rna: 0.1541 loss_x_val_protein: 0.1453 loss_xShared_val_rna: 0.1524 loss_xShared_val_protein: 0.1484 time: 0.4720s
 Epoch: 3559 loss_train: 0.4453 loss_x_train_rna: 0.0974 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.098

 Epoch: 3584 loss_train: 0.4452 loss_x_train_rna: 0.0974 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.0982 loss_xShared_train_protein: 0.1245 loss_val: 0.6018 loss_x_val_rna: 0.1542 loss_x_val_protein: 0.1462 loss_xShared_val_rna: 0.1525 loss_xShared_val_protein: 0.1488 time: 0.4701s
 Epoch: 3585 loss_train: 0.4452 loss_x_train_rna: 0.0974 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.0982 loss_xShared_train_protein: 0.1245 loss_val: 0.6012 loss_x_val_rna: 0.1541 loss_x_val_protein: 0.1464 loss_xShared_val_rna: 0.1524 loss_xShared_val_protein: 0.1483 time: 0.4325s
 Epoch: 3586 loss_train: 0.4452 loss_x_train_rna: 0.0974 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.0982 loss_xShared_train_protein: 0.1245 loss_val: 0.6015 loss_x_val_rna: 0.1544 loss_x_val_protein: 0.1463 loss_xShared_val_rna: 0.1524 loss_xShared_val_protein: 0.1484 time: 0.4755s
 Epoch: 3587 loss_train: 0.4452 loss_x_train_rna: 0.0974 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.098

 Epoch: 3612 loss_train: 0.4451 loss_x_train_rna: 0.0974 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.0981 loss_xShared_train_protein: 0.1245 loss_val: 0.6028 loss_x_val_rna: 0.1547 loss_x_val_protein: 0.1471 loss_xShared_val_rna: 0.1526 loss_xShared_val_protein: 0.1484 time: 0.4277s
 Epoch: 3613 loss_train: 0.4451 loss_x_train_rna: 0.0973 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.0981 loss_xShared_train_protein: 0.1245 loss_val: 0.6035 loss_x_val_rna: 0.1548 loss_x_val_protein: 0.1472 loss_xShared_val_rna: 0.1527 loss_xShared_val_protein: 0.1488 time: 0.3903s
 Epoch: 3614 loss_train: 0.4451 loss_x_train_rna: 0.0974 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.0981 loss_xShared_train_protein: 0.1245 loss_val: 0.6027 loss_x_val_rna: 0.1546 loss_x_val_protein: 0.1470 loss_xShared_val_rna: 0.1525 loss_xShared_val_protein: 0.1487 time: 0.4726s
 Epoch: 3615 loss_train: 0.4451 loss_x_train_rna: 0.0973 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.098

 Epoch: 3640 loss_train: 0.4451 loss_x_train_rna: 0.0973 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.0981 loss_xShared_train_protein: 0.1245 loss_val: 0.6051 loss_x_val_rna: 0.1546 loss_x_val_protein: 0.1480 loss_xShared_val_rna: 0.1527 loss_xShared_val_protein: 0.1498 time: 0.4644s
 Epoch: 3641 loss_train: 0.4450 loss_x_train_rna: 0.0973 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.0981 loss_xShared_train_protein: 0.1245 loss_val: 0.6046 loss_x_val_rna: 0.1546 loss_x_val_protein: 0.1477 loss_xShared_val_rna: 0.1527 loss_xShared_val_protein: 0.1496 time: 0.4708s
 Epoch: 3642 loss_train: 0.4450 loss_x_train_rna: 0.0973 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.0981 loss_xShared_train_protein: 0.1245 loss_val: 0.6044 loss_x_val_rna: 0.1546 loss_x_val_protein: 0.1477 loss_xShared_val_rna: 0.1525 loss_xShared_val_protein: 0.1497 time: 0.4338s
 Epoch: 3643 loss_train: 0.4450 loss_x_train_rna: 0.0973 loss_x_train_protein: 0.1244 loss_xShared_train_rna: 0.098

KeyboardInterrupt: 

In [ ]:
with open(os.path.join(logsavepath,'train_loss_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)), 'wb') as output:
    pickle.dump(train_loss, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'train_loss_rna_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)), 'wb') as output:
    pickle.dump(train_loss_rna, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'val_loss_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)), 'wb') as output:
    pickle.dump(val_loss, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'val_loss_rna_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)), 'wb') as output:
    pickle.dump(val_loss_rna, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'train_loss_protein_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)), 'wb') as output:
    pickle.dump(train_loss_protein, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'val_loss_protein_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)), 'wb') as output:
    pickle.dump(val_loss_protein, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'train_lNorm_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)), 'wb') as output:
    pickle.dump(train_lNorm, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'train_lNormRNA_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)), 'wb') as output:
    pickle.dump(train_lNorm_rna, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(logsavepath,'train_lNormprotein_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)), 'wb') as output:
    pickle.dump(train_lNorm_protein, output, pickle.HIGHEST_PROTOCOL)



plt.plot(np.arange(epochs),train_loss_rna)
plt.plot(np.arange(epochs),val_loss_rna)
# plt.plot(np.arange(epochs),train_loss_kl_ep)
plt.legend(['training x recon loss','validation x recon loss','training kl loss'],loc='upper right')
plt.savefig(os.path.join(plotsavepath,'loss_rna'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'.jpg'))
plt.show()



plt.plot(np.arange(epochs),train_loss_protein)
plt.plot(np.arange(epochs),val_loss_protein)
# plt.plot(np.arange(epochs),train_loss_kl_ep)
plt.legend(['training x recon loss','validation x recon loss','training kl loss'],loc='upper right')
plt.savefig(os.path.join(plotsavepath,'loss_protein'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'.jpg'))
plt.show()


plt.plot(np.arange(epochs),train_lNorm)
plt.plot(np.arange(epochs),train_lNorm_rna)
plt.plot(np.arange(epochs),train_lNorm_protein)
# plt.plot(np.arange(epochs),train_loss_kl_ep)
plt.legend(['shared l2','rna l2','protein l2'],loc='upper right')
plt.savefig(os.path.join(plotsavepath,'loss_lNorm'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'.jpg'))
plt.show()



### shared 50, dspecific 30, hidden 1024

In [25]:
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.enabled = True

model_rna = model_lord.fc_decode_l4(nFeatures_rna,sharedSize+dSpecificSize,hiddenSize, dropout)
model_protein = model_lord.fc_decode_l4(nFeatures_protein, sharedSize+dSpecificSize,hiddenSize, dropout)

model_rna.cuda()
model_protein.cuda()

latent_shared=torch.nn.Embedding(rna.shape[0],sharedSize)
latent_rna=torch.nn.Embedding(rna.shape[0],dSpecificSize)
latent_protein=torch.nn.Embedding(rna.shape[0],dSpecificSize)




In [26]:
loadEpoch='3600'
with open(os.path.join(modelsavepath,'latentRNA_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'_ep'+str(loadEpoch)), 'rb') as output:
    latent_rna.weight=pickle.load(output)
with open(os.path.join(modelsavepath,'latentShared_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'_ep'+str(loadEpoch)), 'rb') as output:
    latent_shared.weight=pickle.load( output)
with open(os.path.join(modelsavepath,'latentprotein_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'_ep'+str(loadEpoch)), 'rb') as output:
    latent_protein.weight=pickle.load(output)
latent_rna.weight.requires_grad=False
latent_shared.weight.requires_grad=False
latent_protein.weight.requires_grad=False

model_rna.load_state_dict(torch.load(os.path.join(modelsavepath,str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'_ep'+str(loadEpoch)+'_rna.pt')))
model_protein.load_state_dict(torch.load(os.path.join(modelsavepath,str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed)+'_ep'+str(loadEpoch)+'_protein.pt')))
model_rna.eval()
model_protein.eval()


fc_decode_l4(
  (fc0): FC(
    (linearlayer): Linear(in_features=80, out_features=1024, bias=False)
    (batchnormlayer): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): FC(
    (linearlayer): Linear(in_features=1024, out_features=1024, bias=False)
    (batchnormlayer): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc2): FC(
    (linearlayer): Linear(in_features=1024, out_features=1024, bias=False)
    (batchnormlayer): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc3): FC(
    (linearlayer): Linear(in_features=1024, out_features=110, bias=False)
  )
)

In [27]:
#plot latent
n_neighbors=10
min_dist=0.25

reducer_sharedlatent = umap.UMAP(n_neighbors=n_neighbors,min_dist=min_dist,random_state=3).fit(latent_shared.weight[trainIdx_all])

In [28]:
def plotembeddingbyCT(ctlist,savename,excludelist,embedding,savepath,plotname,plotdimx=0,plotdimy=1,savenameAdd='',img=None,ncolors=None,colorseq=None,s=2.5):
    
    celltypes=np.unique(ctlist)
    if ncolors is None:
        colortest=sns.color_palette("husl", celltypes.size)
        if colorseq is None:
            colorseq=np.arange(celltypes.size)
    else:
        colortest=sns.color_palette("husl", ncolors)
        if colorseq is None:
            colorseq=np.arange(ncolors)

    
    fig, ax = plt.subplots(dpi=400)
    if not img is None:
        plt.imshow(img)
    for ct in celltypes:
        if ct in excludelist:
            continue
        idx=(ctlist==ct)
        if not img is None:
            ax.scatter(
                embedding[idx, plotdimy],
                embedding[idx, plotdimx],
                color=colortest[colorseq[int(ct)]],label=ct,s=1.5,alpha=0.5
                )
        else:
            ax.scatter(
                embedding[idx, plotdimx],
                embedding[idx, plotdimy],
                color=colortest[colorseq[int(ct)]],label=ct,s=s,alpha=1
                )

    plt.gca().set_aspect('equal', 'datalim')
    fig.set_figheight(5)
    fig.set_figwidth(5)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                     box.width, box.height * 0.9])

    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
              fancybox=True,ncol=5, shadow=True,prop={'size': 6})
    plt.title(plotname+' embedding', fontsize=12)
    plt.savefig(os.path.join(savepath,savename+savenameAdd+'.pdf'))
    
    plt.close('all')
    
def plotembeddingbyCT_str(ctlist,savename,excludelist,embedding,savepath,plotname,plotdimx=0,plotdimy=1,savenameAdd=''):
    celltypes=np.unique(ctlist)
    celltypes_dict={}
    idx=0
    for ct in celltypes:
        celltypes_dict[ct]=idx
        idx+=1
        
    colortest=sns.color_palette("husl", celltypes.size)
    fig, ax = plt.subplots(dpi=400)
    for ct in celltypes:
        if ct in excludelist:
            continue
        idx=(ctlist==ct)
        ax.scatter(
            embedding[idx, plotdimx],
            embedding[idx, plotdimy],
            color=colortest[celltypes_dict[ct]],label=ct,s=1,alpha=0.5
#             color=colortest[int(ct)],label=ct,s=1.5,alpha=0.5
            )

    plt.gca().set_aspect('equal', 'datalim')
    fig.set_figheight(6)
    fig.set_figwidth(6)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.3,
                     box.width, box.height * 0.7])
    # Put a legend below current axis
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), 
              fancybox=True, ncol=3, shadow=True, prop={'size': 6})
    plt.title(plotname, fontsize=12)
    plt.savefig(os.path.join(savepath,savename+savenameAdd+'.pdf'))
    
    plt.close('all')
    
    gc.collect()

In [30]:
ct_top3=np.copy(snl111_rna.obs['cell_types'][trainIdx_all])
for ct in np.unique(ct_top3):
    if ct=='Activated CD4 T':
        ct_top3[ct_top3==ct]='CD4+'
    if ct=='B doublets' or ct=='Low quality B cells':
        ct_top3[ct_top3==ct]='B cells'
    if ct=='CD122+ CD8 T':
        ct_top3[ct_top3==ct]='CD8+'
    if ct=='CD4 T':
        ct_top3[ct_top3==ct]='CD4+'
    if ct=='CD8 T':
        ct_top3[ct_top3==ct]='CD8+'
    if ct=='Mature B':
        ct_top3[ct_top3==ct]='B cells'
    if ct=='Plasma B':
        ct_top3[ct_top3==ct]='B cells'
    if ct=='Transitional B':
        ct_top3[ct_top3==ct]='B cells'
    if ct=='MZ B':
        ct_top3[ct_top3==ct]='B cells'
    if ct=='B1 B':
        ct_top3[ct_top3==ct]='B cells'
    if ct=='Ifit3-high B':
        ct_top3[ct_top3==ct]='B cells'
    if ct=='Ifit3-high CD4 T':
        ct_top3[ct_top3==ct]='CD4+'
    if ct=='Ifit3-high CD8 T':
        ct_top3[ct_top3==ct]='CD8+'
    if 'mono' in ct or 'DC' in ct:
        ct_top3[ct_top3==ct]='myeloid'
    if ct=='MZ/Marco-high macrophages' or ct=='Red-pulp macrophages':
        ct_top3[ct_top3==ct]='myeloid'
    elif ct not in ['Neutrophils']:
        ct_top3[ct_top3==ct]='others'
plotembeddingbyCT_str(ct_top3,'celltypeTop3_shared_latent',[],reducer_sharedlatent.embedding_,plotsavepath,'cell type',plotdimx=0,plotdimy=1,savenameAdd='_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed))
plotembeddingbyCT_str(np.array(snl111_rna.obs['batch_indices'][trainIdx_all]),'batch_shared_latent',[],reducer_sharedlatent.embedding_,plotsavepath,'batch',plotdimx=0,plotdimy=1,savenameAdd='_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed))


In [34]:
palette = sns.color_palette("husl", n_colors=2)  

# Print the color codes in hex format
hex_colors = palette.as_hex()  
for color in hex_colors:
    print(color)

#f77189
#36ada4


In [32]:
color

(0.9677975592919913, 0.44127456009157356, 0.5358103155058701)

In [ ]:
plotembeddingbyCT_str(ct_top3,'celltypeTop3_protein_specific_latent',[],reducer_proteinspecific.embedding_,plotsavepath,'cell type',plotdimx=0,plotdimy=1,savenameAdd='_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed))
plotembeddingbyCT_str(np.array(snl111_rna.obs['batch_indices'][trainIdx_all]),'batch_protein_specific_latent',[],reducer_proteinspecific.embedding_,plotsavepath,'batch',plotdimx=0,plotdimy=1,savenameAdd='_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed))


In [ ]:
plotembeddingbyCT_str(ct_top3,'celltypeTop3_rna_specific_latent',[],reducer_rnaspecific.embedding_,plotsavepath,'cell type',plotdimx=0,plotdimy=1,savenameAdd='_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed))
plotembeddingbyCT_str(np.array(snl111_rna.obs['batch_indices'][trainIdx_all]),'batch_rna_specific_latent',[],reducer_rnaspecific.embedding_,plotsavepath,'batch',plotdimx=0,plotdimy=1,savenameAdd='_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed))


In [26]:

rna_both=snl111_both[:,snl111_both.var["hvg_encode"]].X.toarray()
rna_both=np.log(rna_both+1/2)
rna_both=(rna_both-np.min(rna_both,axis=1,keepdims=True))/(np.max(rna_both,axis=1,keepdims=True)-np.min(rna_both,axis=1,keepdims=True))

n_neighbors=10
min_dist=0.25

pca=PCA(n_components=30)
pca_rnaboth=pca.fit_transform(rna_both)
reducer_rnaboth = umap.UMAP(n_neighbors=n_neighbors,min_dist=min_dist,random_state=3).fit(pca_rnaboth)
plotembeddingbyCT_str(ct_top3,'celltypeTop_rnaboth',[],reducer_rnaboth.embedding_,plotsavepath,'cell type',plotdimx=0,plotdimy=1,savenameAdd='_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed))
plotembeddingbyCT_str(np.array(snl111_both.obs['batch_indices']),'batch_rnaboth',[],reducer_rnaboth.embedding_,plotsavepath,'batch',plotdimx=0,plotdimy=1,savenameAdd='_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed))


In [25]:
ct_top.shape

(16828,)

In [27]:
#preprocess protein
keep_pro_111 = np.array(
    [not p.startswith("HTO") for p in snl111_both.uns["protein_names"]]
)

snl111_both.obsm["protein_expression"] = snl111_both.obsm["protein_expression"][
    :, keep_pro_111
]
snl111_both.uns["protein_names"] = snl111_both.uns["protein_names"][keep_pro_111]

protein_both=np.array(snl111_both.obsm["protein_expression"])

In [28]:
#preprocess
protein_both=np.log(protein_both+1/2)
protein_both=(protein_both-np.min(protein_both,axis=1,keepdims=True))/(np.max(protein_both,axis=1,keepdims=True)-np.min(protein_both,axis=1,keepdims=True))


In [ ]:
n_neighbors=10
min_dist=0.25

pca=PCA(n_components=30)
pca_proteinboth=pca.fit_transform(protein_both)
reducer_proteinboth = umap.UMAP(n_neighbors=n_neighbors,min_dist=min_dist,random_state=3).fit(pca_proteinboth)
plotembeddingbyCT_str(ct_top3,'celltypeTop_proteinboth',[],reducer_proteinboth.embedding_,plotsavepath,'cell type',plotdimx=0,plotdimy=1,savenameAdd='_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed))
plotembeddingbyCT_str(np.array(snl111_both.obs['batch_indices']),'batch_proteinboth',[],reducer_proteinboth.embedding_,plotsavepath,'batch',plotdimx=0,plotdimy=1,savenameAdd='_'+str(sharedSize)+'_'+str(dSpecificSize)+'_'+str(hiddenSize)+'_'+str(seed))
